In [10]:
import json
import requests
import time
import pandas as pd
import csv

In [2]:
def read_api_key(file_name="api_key.txt"):
    try:
        with open(file_name, "r") as file:
            api_key = file.read().strip()
        return api_key
    except FileNotFoundError:
        print(f"Error: {file_name} not found.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the API key: {e}")
        return None

In [3]:
api_key = read_api_key()
if api_key:
    print("API key successfully loaded.")
else:
    print("Failed to load API key.")

API key successfully loaded.


In [4]:
REQUEST_DELAY = 1.2

In [5]:
def fetch_data_with_retries(url, max_retries=3, delay=REQUEST_DELAY):
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print("Rate limit exceeded. Retrying...")
                time.sleep(delay * 2)
            else:
                print(f"Failed request to {url}. Status code: {response.status_code}")
                return None
        except requests.RequestException as e:
            print(f"Request error: {e}")
        retries += 1
        time.sleep(delay)
    return None

In [6]:
grandmaster_url = f'https://kr.api.riotgames.com/tft/league/v1/grandmaster?queue=RANKED_TFT&api_key={api_key}'
grandmaster_data = fetch_data_with_retries(grandmaster_url)

summoner_ids = []
if grandmaster_data:
    summoner_ids = [entry['summonerId'] for entry in grandmaster_data['entries']]

match_ids = []

In [7]:
for summoner_id in summoner_ids:
    summoner_url = f'https://kr.api.riotgames.com/tft/summoner/v1/summoners/{summoner_id}?api_key={api_key}'
    summoner_data = fetch_data_with_retries(summoner_url)
    
    if summoner_data:
        puuid = summoner_data['puuid']
        
        match_url = f'https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={api_key}'
        match_data = fetch_data_with_retries(match_url)
        
        if match_data:
            match_ids.extend(match_data)
        time.sleep(REQUEST_DELAY)

In [8]:
unique_match_ids = list(set(match_ids))

print(f'Total unique match IDs for Grandmaster players in Korea: {len(unique_match_ids)}')

Total unique match IDs for Grandmaster players in Korea: 6598


In [14]:
def save_list_to_csv(data, file_name):
    if not data:
        print("Data is empty, nothing to save.")
        return
    
    with open(file_name, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for item in data:
            writer.writerow([item])

In [15]:
file_name = "match_id_grandmaster_20241223_1403.csv"
save_list_to_csv(unique_match_ids, file_name)
print(f"Data saved to {file_name}")

Data saved to match_id_grandmaster_20241223_1403.csv
